# 0. Load Data

In [1]:
import math 
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Isolation Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# VAE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics

In [2]:
cell_ids = pd.read_csv("data/cells/cell_ids.csv",header=None)
cell_ids = [x for lst in cell_ids.values.tolist() for x in lst]
cell_ids

[2.2265366483183206e+17,
 7.315874467898523e+16,
 9.10621795573706e+16,
 3.8447326973958944e+17,
 4.262775046883192e+17]

In [3]:
cells = []
for cell_id in cell_ids:
    file_name = "cell_"+str(cell_id)+'.csv'
    cells.append(pd.read_csv(f"data/cells/{file_name}"))

In [4]:
cells[0].head()

,index,cell_id,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,PDCP_SDU_Volume_DL,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,Intra_eNB_HO_SR_total_RATIO,...,DCR_LTE_RATIO,CSSR_LTE_RATIO,LTE_INTER_ENODEB_HOSR_RATIO,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,DL_THROUGHPUT_RATIO,E_RAB_DR_RATIO,HOUR
0,2021-05-09 00:00:00,2.226537e+17,3.779737e+10,3.947172e+09,15.0,3.779737e+10,4727.0,0.809859,0.992427,0.809859,...,0.001761,0.996041,0.400000,0.770642,0.963636,0.963636,0.400000,82.550417,0.001761,0
1,2021-05-09 01:00:00,2.226537e+17,3.684898e+10,4.088752e+09,6.0,3.684898e+10,3076.0,0.886792,0.993288,0.886792,...,0.002468,0.995465,0.500000,0.842105,1.000000,1.000000,0.500000,80.531811,0.002468,1
2,2021-05-09 02:00:00,2.226537e+17,3.292677e+10,5.016897e+09,8.0,3.292677e+10,3501.0,0.938356,0.994664,0.938356,...,0.003077,0.996044,0.375000,0.931624,1.000000,0.966667,0.375000,71.885311,0.003077,2
3,2021-05-09 03:00:00,2.226537e+17,3.021547e+10,5.139107e+09,9.0,3.021547e+10,2275.0,0.860215,0.994819,0.860215,...,0.001721,0.995920,0.777778,0.816901,0.947368,0.947368,0.777778,65.977430,0.001721,3
4,2021-05-09 04:00:00,2.226537e+17,3.082176e+10,4.250716e+09,17.0,3.082176e+10,2178.0,0.840426,0.995952,0.840426,...,0.002213,0.995628,0.764706,0.794521,1.000000,1.000000,0.764706,67.324805,0.002213,4


# 1. Isolation Forest

In [5]:
model=IsolationForest(n_estimators=150, 
                      max_samples ='auto', 
                      max_features=1)

In [6]:
df_cell = cells[0].drop(['cell_id','index'],axis=1)
model.fit(df_cell)
# score_samples = - score  
IF_score = -1 * model.score_samples(df_cell)

In [ ]:
df_cell['IF_score'] = IF_score
df_cell['IF_score'].describe()

# 2. VAE

[keras - Variational AutoEncoder 官方代码范例(复杂)](https://keras.io/examples/generative/vae/)

[keras - AE/VAE 相对简单的代码构建](https://blog.keras.io/building-autoencoders-in-keras.html)

[keras - 中等程度AVE 但是有点乱](https://github.com/keras-team/keras/blob/2c8d1d03599cc03243bce8f07ed9c4a3d5f384f9/examples/variational_autoencoder.py)







[中文VAE理论+代码](https://blog.csdn.net/weixin_37737254/article/details/102920263)

[keras - 函数式API](https://keras.io/guides/functional_api/)

[keras - Conv1D](https://keras-zh.readthedocs.io/layers/convolutional/)


## 2.1 Build model

In [4]:
################  Standarlize & Split data  #################
df_cell = cells[0].drop(['cell_id','index'],axis=1)
df_cell = np.expand_dims(df_cell,axis=1)
x_train, x_test = train_test_split(df_cell)

x_train = StandardScaler().fit_transform(x_train)
x_test  = StandardScaler().fit_transform(x_test)

In [5]:
################ Define sampling function #################
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

In [6]:
################ Build Encoder #################
original_dim = df_cell.shape[2]
intermediate_dim = 32
latent_dim = 2

# encoder input
inputs = keras.Input(shape=(1,original_dim))
x = layers.Conv1D(filters=intermediate_dim, kernel_size=3, padding="same", activation="relu")(inputs)
x = layers.Dropout(rate=0.2)(x)
x = layers.Flatten()(x)

# the output of encoder
z_mean      = layers.Dense(latent_dim, name="z_mean")(x)
z_log_sigma = layers.Dense(latent_dim, name="z_log_var")(x)
z           = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 24)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 1, 32)        2336        ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 1, 32)        0           ['conv1d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 32)           0           ['dropout[0][0]']                
                                                                                            

In [7]:
################ Build Decoder #################

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(1*intermediate_dim, activation='relu')(latent_inputs)
x = layers.Reshape((1,intermediate_dim))(x)
x = layers.Conv1DTranspose(filters=intermediate_dim, kernel_size=3, padding="same", activation="relu")(x)
x = layers.Dropout(rate=0.2)(x)
outputs = layers.Conv1DTranspose(filters=original_dim, kernel_size=3, padding="same", activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 32)                96        
                                                                 
 reshape (Reshape)           (None, 1, 32)             0         
                                                                 
 conv1d_transpose (Conv1DTra  (None, 1, 32)            3104      
 nspose)                                                         
                                                                 
 dropout_1 (Dropout)         (None, 1, 32)             0         
                                                                 
 conv1d_transpose_1 (Conv1DT  (None, 1, 24)            2328      
 ranspose)                                                 

In [8]:
################ Instantiate VAE model ################# 
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

################     Loss Function      ################# 
reconstruction_loss = keras.losses.mean_squared_error(inputs, outputs)
# reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + 0.0001*kl_loss) #调整比例？？
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')


ps: weight of kl_loss?    

ref1.[theoretical insight](https://stats.stackexchange.com/questions/332179/how-to-weight-kld-loss-vs-reconstruction-loss-in-variational-auto-encoder)

ref2.[a more specific example](https://stats.stackexchange.com/questions/341954/balancing-reconstruction-vs-kl-loss-variational-autoencoder)

In [9]:
# Train model
vae.fit(x_train, x_train,
        epochs=100,
        batch_size=32,
        validation_data=(x_test, x_test))

Epoch 1/100
39/39 [==============================] - 1s 8ms/step - loss: 1.2529 - val_loss: 1.1129
Epoch 2/100
39/39 [==============================] - 0s 3ms/step - loss: 1.1206 - val_loss: 0.9210
Epoch 3/100
39/39 [==============================] - 0s 3ms/step - loss: 1.0054 - val_loss: 0.8732
Epoch 4/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9729 - val_loss: 0.8558
Epoch 5/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9569 - val_loss: 0.8429
Epoch 6/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9461 - val_loss: 0.8334
Epoch 7/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9385 - val_loss: 0.8268
Epoch 8/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9342 - val_loss: 0.8235
Epoch 9/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9306 - val_loss: 0.8204
Epoch 10/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9276 - val_loss: 0.8173
Epoch 11/

39/39 [==============================] - 0s 3ms/step - loss: 0.8462 - val_loss: 0.7457
Epoch 84/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8479 - val_loss: 0.7452
Epoch 85/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8446 - val_loss: 0.7456
Epoch 86/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8430 - val_loss: 0.7440
Epoch 87/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8443 - val_loss: 0.7447
Epoch 88/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8442 - val_loss: 0.7448
Epoch 89/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8425 - val_loss: 0.7449
Epoch 90/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8428 - val_loss: 0.7448
Epoch 91/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8437 - val_loss: 0.7448
Epoch 92/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8474 - val_loss: 0.7434
Epoch 93/100


In [11]:
vae.save("model/2_VAE/cell0/")

INFO:tensorflow:Assets written to: model/2_VAE/cell0\assets


## 2.2 [Detect Anomaly](https://towardsdatascience.com/hands-on-anomaly-detection-with-variational-autoencoders-d4044672acd5)
Detect anomaly by finding those who have a high reconstruction loss.


Steps:
1. Measure error between the original train (clean/normal) set and the output of the model, and generate an error vector representing the error term of each sample.

2. Find a relatively extreme value on that vector to use as your error threshold.

3. Run the model over the test or real data, in which anomalies are probably mixed with normal data.

4. Measure the reconstruction error and mark samples that exhibit an error term higher than the error threshold as anomalies.

In [16]:
def error(original,reconstructed):
    return np.mean(np.sum((original-reconstructed)**2,axis=1),axis=1)

In [37]:
from sklearn.preprocessing import MinMaxScaler
x_train_pred = vae.predict(x_train)
x_test_pred  = vae.predict(x_test)

x_test_loss = error(x_test,x_test_pred)
x_test_loss[:20]

array([0.35374606, 0.52598968, 1.54053563, 2.1078484 , 0.1651465 ,
       1.47905311, 1.15083688, 0.35015205, 0.43852304, 0.88559885,
       1.10095315, 1.09408546, 1.35447138, 0.30294214, 0.75938145,
       0.34638426, 0.64857334, 0.36229537, 0.78489424, 1.48647557])

In [38]:
#just to show the 
MinMaxScaler().fit_transform(np.array(x_test_loss).reshape(-1, 1))[:20]

array([[0.01711937],
       [0.02734142],
       [0.08755112],
       [0.12121912],
       [0.00592666],
       [0.08390235],
       [0.06442389],
       [0.01690608],
       [0.02215059],
       [0.04868295],
       [0.06146346],
       [0.06105589],
       [0.07650887],
       [0.01410434],
       [0.04119239],
       [0.01668248],
       [0.03461633],
       [0.01762674],
       [0.04270649],
       [0.08434285]])

In [ ]:
x_test_loss.sha

In [ ]:
def anomalyPercent(model, df_test):
    """
    Calculate the percentage of anomalies in the testset
    
    """
    df_pred = df_test.copy()
    df_pred['anomaly'] = model.predict(df_test)
    pct  = (df_pred['anomaly']==-1).sum()/len(df_test)
    
#     # map 1 -> 0, -1 -> 1
#     reset_value = lambda x: 1 if x==-1 else 0
#     df_test['anomaly'] = df_test['anomaly'].map(reset_value)
    
    return pct, df_pred

In [ ]:
def anomaly_visualization(df):
    """
    Visualize anomalies in 2D/3D scatter plot
    
    INPUT
    @df : a dataframe where the prediction result is in the 'anomaly' column
    
    """
    index = df.index
    outlier_index = list(df[df['anomaly']==-1].index)
    
    # nomalize the metrics
    X = StandardScaler().fit_transform(df.drop('anomaly',axis=1))
    
    # recude the dimension for visualization
    X_3d = pd.DataFrame(PCA(3).fit_transform(X),index=index)
    X_3d_outliers = X_3d.reindex(outlier_index)
    X_2d = pd.DataFrame(PCA(2).fit_transform(X),index=index)
    X_2d_outliers = X_2d.reindex(outlier_index)
    
    # plot
    fig = plt.figure(figsize=(10,5))
    # 3D
    ax1 = fig.add_subplot(121,  projection='3d')
    ax1.scatter(X_3d.loc[:,0],X_3d.loc[:,1],X_3d.loc[:,2],
                s=4,lw=1,label="inliers",c="green",alpha=0.5)
    ax1.scatter(X_3d_outliers.loc[:,0], X_3d_outliers.loc[:,1], X_3d_outliers.loc[:,2],
               s=60,marker="x",lw=2,label="outliers",c="red")
    ax1.legend()
    ax1.set_title("Isolation Prediction (3D)")
    # 2D
    ax2 = fig.add_subplot(133)
    ax2.scatter(X_2d.loc[:,0],X_2d.loc[:,1],c="green",
                s=5,label="inliers",alpha=0.8)
    ax2.scatter(X_2d_outliers.iloc[:,0], X_2d_outliers.iloc[:,1],
                s=60,marker="x",lw=2,c='red',label="outliers")
    ax2.legend()
    ax2.set_title("Isolation Prediction (2D)")
    plt.show()
    

In [ ]:
for i in range(len(list_cells)):
    # get the dataframe of a certain cell
    df_c = list_cells[i].drop(['cell_id','index'], axis=1)
    
    # split train set and test set 
    df_train, df_test = train_test_split(df_c)
    # train model 
    model.fit(df_train)
    pct, df_pred = anomalyPercent(model, df_test)
    print("="*75)
    print(f"cell id: {cell_ids[i]}")
    print("Percentage of outliers: %.3f "%pct)
    anomaly_visualization(df_pred)
    print("\n\n")